In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/StudentPerformanceFactors.csv')
df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


In [2]:
# Average exam score by motivation level
df.groupby('Motivation_Level', observed=True)['Exam_Score'].mean()

Motivation_Level
High      67.704321
Low       66.752194
Medium    67.330648
Name: Exam_Score, dtype: float64

In [ ]:
# Multiple aggregations
df.groupby('Motivation_Level', observed=True)['Exam_Score'].describe()



,count,mean,std,min,25%,50%,75%,max
Motivation_Level,,,,,,,,
High,1319.0,67.704321,3.876966,57.0,65.0,67.0,70.0,98.0
Low,1937.0,66.752194,3.958139,57.0,64.0,67.0,69.0,101.0
Medium,3351.0,67.330648,3.827303,55.0,65.0,67.0,69.0,100.0


In [4]:
# Multiple columns and aggregations
df.groupby(['Motivation_Level', 'Internet_Access'], observed=True)['Exam_Score'].mean()

Motivation_Level  Internet_Access
High              No                 66.967391
                  Yes                67.759576
Low               No                 66.277778
                  Yes                66.795493
Medium            No                 66.542857
                  Yes                67.392788
Name: Exam_Score, dtype: float64

In [5]:
# agg() - Multiple and Named Aggregations 
# Multiple functions with agg() 

df.groupby('Motivation_Level', observed=True)['Exam_Score'].agg(['mean', 'sum', 'count', 'median', 'min', 'max'])

,mean,sum,count,median,min,max
Motivation_Level,,,,,,
High,67.704321,89302,1319,67.0,57,98
Low,66.752194,129299,1937,67.0,57,101
Medium,67.330648,225625,3351,67.0,55,100


In [6]:
# Diffrent aggrations per Column 

df.groupby(['Family_Income'], observed=True).agg({
    'Exam_Score': 'mean',
    'Hours_Studied': 'mean', 
    'Attendance': 'mean',
    'Tutoring_Sessions': 'sum'
})

,Exam_Score,Hours_Studied,Attendance,Tutoring_Sessions
Family_Income,,,,
High,67.842396,19.903861,79.907801,1910
Low,66.848428,19.933009,80.215195,3975
Medium,67.334959,20.051763,79.772318,3984


In [7]:
# named Aggregations (Modern Approach)

result = df.groupby('Motivation_Level', observed=True).agg(
    avg_score = ('Exam_Score', 'mean'),
    max_score = ('Exam_Score', 'max'),
    avg_hours = ('Hours_Studied', 'mean'),
    student_count = ('Exam_Score', 'count')
)

result

,avg_score,max_score,avg_hours,student_count
Motivation_Level,,,,
High,67.704321,98,19.730099,1319
Low,66.752194,101,19.956118,1937
Medium,67.330648,100,20.082960,3351


In [8]:
# reset index , to make it a regular dataframe
result = result.reset_index()
result

,Motivation_Level,avg_score,max_score,avg_hours,student_count
0,High,67.704321,98,19.730099,1319
1,Low,66.752194,101,19.956118,1937
2,Medium,67.330648,100,20.082960,3351


In [9]:
# .transform() 

# Broadcasting group stats

# Add group mean as anew column (same number of rows)
df['Group_Avg_Score'] = df.groupby('Motivation_Level', observed=True)['Exam_Score'].transform('mean')
df[['Motivation_Level', 'Exam_Score', 'Group_Avg_Score']].head()

,Motivation_Level,Exam_Score,Group_Avg_Score
0,Low,67,66.752194
1,Low,61,66.752194
2,Medium,74,67.330648
3,Medium,71,67.330648
4,Medium,70,67.330648


In [10]:
# How much each student deviates from their group average?
df['Score_vs_Group'] = df['Exam_Score'] - df['Group_Avg_Score']
df[['Motivation_Level', 'Exam_Score', 'Group_Avg_Score', 'Score_vs_Group']].head(10)

,Motivation_Level,Exam_Score,Group_Avg_Score,Score_vs_Group
0,Low,67,66.752194,0.247806
1,Low,61,66.752194,-5.752194
2,Medium,74,67.330648,6.669352
3,Medium,71,67.330648,3.669352
4,Medium,70,67.330648,2.669352
5,Medium,71,67.330648,3.669352
6,Low,67,66.752194,0.247806
7,Medium,66,67.330648,-1.330648
8,High,69,67.704321,1.295679
9,Medium,72,67.330648,4.669352


In [11]:
# pivote_table() - Excel Style Pivoting

# Average exam score : rows = MOtivation, columsn = Internet Access

pd.pivot_table(
    df,
    values='Exam_Score',
    index='Motivation_Level',
    columns='Internet_Access',
    aggfunc='mean'
).round(1)

Internet_Access,No,Yes
Motivation_Level,,
High,67.0,67.8
Low,66.3,66.8
Medium,66.5,67.4


In [12]:
# pivot with multiple aggrations
pd.pivot_table(
    df,
    values='Exam_Score',
    index='Family_Income',
    columns='Motivation_Level',
    aggfunc=['mean', 'count']
).round(1)

mean              count            
Motivation_Level  High   Low Medium  High  Low Medium
Family_Income                                        
High              68.3  67.5   67.9   250  374    645
Low               67.2  66.3   67.0   510  803   1359
Medium            67.9  66.8   67.4   559  760   1347

In [13]:
# margins (Totals)
# Add row and column totals
pd.pivot_table(
    df,
    values='Exam_Score',
    index='Motivation_Level',
    columns='Gender',
    aggfunc='mean',
    margins=True,
    margins_name='Overall'
).round(1)

Gender,Female,Male,Overall
Motivation_Level,,,
High,67.5,67.8,67.7
Low,66.8,66.7,66.8
Medium,67.4,67.3,67.3
Overall,67.2,67.2,67.2


In [14]:
# crosstab()

# Count of students by Motivation and Internet Access
pd.crosstab(df['Motivation_Level'], df['Internet_Access'])

Internet_Access,No,Yes
Motivation_Level,,
High,92,1227
Low,162,1775
Medium,245,3106


In [15]:
# Normalize to show percentage
pd.crosstab(df['Motivation_Level'], df['Internet_Access'], normalize='index').round(2) * 100

Internet_Access,No,Yes
Motivation_Level,,
High,7.0,93.0
Low,8.0,92.0
Medium,7.0,93.0


In [16]:
# What factors most impact student exam performance ? 


# ============================================
# ANALYTICS PIPELINE
# ============================================

# Question: What factors most impact exam scores?

# Step 1 : Average score by key factors
factors = ['Motivation_Level', 'Internet_Access', 'Family_Income', 'Parental_Involvement', 'Teacher_Quality', 'School_Type']

print("=== AVERAGE EXAM SCORE BY FACTOR ===")
for factor in factors:
    avg = df.groupby(factor, observed=True)['Exam_Score'].mean().round(1)
    spread = avg.max() - avg.min()
    print(f"{factor} (spread: {spread:.1f} pts)")
    print(avg.to_string())
    print("")

=== AVERAGE EXAM SCORE BY FACTOR ===
Motivation_Level (spread: 0.9 pts)
Motivation_Level
High      67.7
Low       66.8
Medium    67.3

Internet_Access (spread: 0.8 pts)
Internet_Access
No     66.5
Yes    67.3

Family_Income (spread: 1.0 pts)
Family_Income
High      67.8
Low       66.8
Medium    67.3

Parental_Involvement (spread: 1.7 pts)
Parental_Involvement
High      68.1
Low       66.4
Medium    67.1

Teacher_Quality (spread: 0.9 pts)
Teacher_Quality
High      67.7
Low       66.8
Medium    67.1

School_Type (spread: 0.1 pts)
School_Type
Private    67.3
Public     67.2



In [17]:
# step 2 : Detailed breakdown of top factors 
summary = df.groupby('Motivation_Level', observed=True).agg(
    avg_score = ('Exam_Score', 'mean'),
    avg_hours = ('Hours_Studied', 'mean'),
    avg_attendance = ('Attendance', 'mean'),
    count = ('Exam_Score', 'count')
).round(1)

print("=== DETAILED MOTIVATION ANALYSIS ===")
summary

=== DETAILED MOTIVATION ANALYSIS ===


,avg_score,avg_hours,avg_attendance,count
Motivation_Level,,,,
High,67.7,19.7,79.7,1319
Low,66.8,20.0,79.9,1937
Medium,67.3,20.1,80.1,3351


In [18]:
# Step 3 : Cross factor analysis 
print("=== CROSS FACTOR : MOTIVATRION x INTERNET ACCESS ===")
cross = pd.pivot_table(
    df, values = 'Exam_Score',
    index = 'Motivation_Level',
    columns = 'Internet_Access',
    aggfunc = 'mean', margins = True,
    margins_name = 'All'
).round(1)
print(cross)

=== CROSS FACTOR : MOTIVATRION x INTERNET ACCESS ===
Internet_Access     No   Yes   All
Motivation_Level                  
High              67.0  67.8  67.7
Low               66.3  66.8  66.8
Medium            66.5  67.4  67.3
All               66.5  67.3  67.2
